# Practical 10

# Create functions to build a temporal datasets for transactional data

Today we will be using a dataset that "contains household level transactions over two years from a group of 2,500 households who are frequent shoppers at a retailer.".

The full documentation can be found on Moodle. Note the full data source [is avaliable onine](https://www.dunnhumby.com/careers/engineering/sourcefiles). 

The table of interest for today is:<br>
`transaction_data`

Please look up what the fields mean in the documentation.


In [8]:
#Import data as pandas dataframe
import pandas as pd
data = pd.read_csv('transaction_data.csv', header = 0)

In [9]:
#check the first 10 colums of data
data.head(10)

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
5,2375,26984851516,1,826249,2,1.98,364,-0.60,1642,1,0.0,0.0
6,2375,26984851516,1,1043142,1,1.57,364,-0.68,1642,1,0.0,0.0
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0
8,2375,26984851516,1,1102651,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,6423775,1,2.00,364,-0.79,1642,1,0.0,0.0


# Part A
## Task 1a
Create a temporal dataset (pandas dataframe) that has:
* a Reference Date of day: MAX(day) - 7<br>
* a tumbling window of 7 days for 5 periods<br>
* `sum` as the tumbling window aggregate function<br>

The output feature should be the total sales for 7 days after the Reference Date.

In [10]:
# Find the max of DAY using the max() function
data.DAY.max()

711

In [11]:
# Define ref_day as Max(DAY) -7
ref_day = data.DAY.max() -7
ref_day

704

In [12]:
# First, we need to group the data by 'household_key'
data2 = data.groupby('household_key')

In [15]:
#Create a dataframe with the output_feature
df = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > ref_day) & (x['DAY'] <= ref_day+7)]['SALES_VALUE'].sum()).reset_index(name ='output_feature'))


In [16]:
#Following the example above, create other dataframes with the input features f1 - f5
df1 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > ref_day-7) & (x['DAY'] <= ref_day)]['SALES_VALUE'].sum()).reset_index(name ='f1'))
df2 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > ref_day-14) & (x['DAY'] <= ref_day-7)]['SALES_VALUE'].sum()).reset_index(name ='f2'))
df3 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > ref_day-21) & (x['DAY'] <= ref_day-14)]['SALES_VALUE'].sum()).reset_index(name ='f3'))
df4 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > ref_day-28) & (x['DAY'] <= ref_day-21)]['SALES_VALUE'].sum()).reset_index(name ='f4'))
df5 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > ref_day-35) & (x['DAY'] <= ref_day-28)]['SALES_VALUE'].sum()).reset_index(name ='f5'))

In [17]:
# Merge the dataframes together 
df['f1'] = df1['f1']
df['f2'] = df2['f2']
df['f3'] = df3['f3']
df['f4'] = df4['f4']
df['f5'] = df5['f5']

In [18]:
df.head(10)

,household_key,output_feature,f1,f2,f3,f4,f5
0,1,81.29,0.00,176.48,33.41,0.00,79.07
1,2,0.00,0.00,0.00,0.00,0.00,0.00
2,3,0.00,36.38,0.00,0.00,0.00,22.53
3,4,0.00,0.00,0.00,0.00,0.00,0.00
4,5,0.00,12.99,16.98,0.00,0.00,0.00
5,6,95.04,0.00,52.65,64.47,42.40,39.09
6,7,59.98,111.79,86.34,152.71,0.00,123.34
7,8,52.66,144.69,0.00,104.16,88.35,76.28
8,9,0.00,0.00,0.00,31.01,18.09,0.00
9,10,0.00,0.00,0.00,40.88,0.00,0.00


## Task 1b
Convert the code to be a generic function of the form:<br>
get_dataset(rd, tws)
n,b., rd = reference_day, tws = tumbling_window_size 

It should return two objects:<br>
`dataset_input_features`<br>
`dataset_output_features`

In [19]:
def get_dataset(rd, tws):
    #complete the code here
    data2 = data.groupby('household_key')
    df = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > rd) & (x['DAY'] <= rd+tws)]['SALES_VALUE'].sum()).reset_index(name ='output_feature'))
    df1 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > rd-tws) & (x['DAY'] <= rd)]['SALES_VALUE'].sum()).reset_index(name ='f1'))
    df2 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > rd-tws*2) & (x['DAY'] <= rd-tws)]['SALES_VALUE'].sum()).reset_index(name ='f2'))
    df3 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > rd-tws*3) & (x['DAY'] <= rd-tws*2)]['SALES_VALUE'].sum()).reset_index(name ='f3'))
    df4 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > rd-tws*4) & (x['DAY'] <= rd-tws*3)]['SALES_VALUE'].sum()).reset_index(name ='f4'))
    df5 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > rd-tws*5) & (x['DAY'] <= rd-tws*4)]['SALES_VALUE'].sum()).reset_index(name ='f5'))
    df1['f2'] = df2['f2']
    df1['f3'] = df3['f3']
    df1['f4'] = df4['f4']
    df1['f5'] = df5['f5']
    return df1.drop(columns = ['household_key'], inplace = False), df.drop(columns = ['household_key'], inplace = False)
    

In [20]:
X, y = get_dataset(rd = 704, tws= 14 )

In [21]:
# Solution
print('X.head(5)')
print(X.head(5))
print()
print('y.head(5)')
print(y.head(5))

X.head(5)
       f1     f2      f3    f4      f5
0  176.48  33.41   79.07  92.2  107.92
1    0.00   0.00  130.89   0.0    0.00
2   36.38   0.00   22.53   0.0   39.32
3    0.00   0.00    0.00   0.0    0.00
4   29.97   0.00    0.00   0.0    0.00

y.head(5)
   output_feature
0           81.29
1            0.00
2            0.00
3            0.00
4            0.00


## Task 2a
Create a temporal dataset (pandas dataframe) that has:
* a tumbling window of 7 days for 5 periods<br>
* `sum` as the tumbling window aggregate function<br>

The output featue should be the total sales for 14 days after the Reference Date.<br>
Set the Reference Date to create the most recent dataset possible.

In [22]:
# Define the ref_day
ref_day = 711 -14
#create a dataframe with the output_feature
df = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > ref_day) & (x['DAY'] <= ref_day+14)]['SALES_VALUE'].sum()).reset_index(name ='output_feature'))
#Create other dataframes with the input features f1 - f5
df1 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > ref_day-7) & (x['DAY'] <= ref_day)]['SALES_VALUE'].sum()).reset_index(name ='f1'))
df2 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > ref_day-14) & (x['DAY'] <= ref_day-7)]['SALES_VALUE'].sum()).reset_index(name ='f2'))
df3 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > ref_day-21) & (x['DAY'] <= ref_day-14)]['SALES_VALUE'].sum()).reset_index(name ='f3'))
df4 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > ref_day-28) & (x['DAY'] <= ref_day-21)]['SALES_VALUE'].sum()).reset_index(name ='f4'))
df5 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > ref_day-35) & (x['DAY'] <= ref_day-28)]['SALES_VALUE'].sum()).reset_index(name ='f5'))
# Merge the dataframes together 
df['f1'] = df1['f1']
df['f2'] = df2['f2']
df['f3'] = df3['f3']
df['f4'] = df4['f4']
df['f5'] = df5['f5']

In [23]:
df.head(10)

,household_key,output_feature,f1,f2,f3,f4,f5
0,1,81.29,176.48,33.41,0.00,79.07,0.00
1,2,0.00,0.00,0.00,0.00,0.00,130.89
2,3,36.38,0.00,0.00,0.00,22.53,0.00
3,4,0.00,0.00,0.00,0.00,0.00,0.00
4,5,12.99,16.98,0.00,0.00,0.00,0.00
5,6,95.04,52.65,64.47,42.40,39.09,90.56
6,7,171.77,86.34,152.71,0.00,123.34,0.00
7,8,197.35,0.00,104.16,88.35,76.28,191.48
8,9,0.00,0.00,31.01,18.09,0.00,11.39
9,10,0.00,0.00,40.88,0.00,0.00,29.52


## Task 2b

Convert the code to be a generic function of the form:<br>
get_dataset( rd, tws, ows)

n,b., rd = reference_day, tws = tumbling_window_size, ows = output_window_size

It should return two objects:<br>
`dataset_input_features`<br>
`dataset_output_features`

In [24]:
def get_dataset(rd, tws, ows):
    #complete the code here
    data2 = data.groupby('household_key')
    df = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > rd) & (x['DAY'] <= rd+ows)]['SALES_VALUE'].sum()).reset_index(name ='output_feature'))
    df1 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > rd-tws) & (x['DAY'] <= rd)]['SALES_VALUE'].sum()).reset_index(name ='f1'))
    df2 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > rd-tws*2) & (x['DAY'] <= rd-tws)]['SALES_VALUE'].sum()).reset_index(name ='f2'))
    df3 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > rd-tws*3) & (x['DAY'] <= rd-tws*2)]['SALES_VALUE'].sum()).reset_index(name ='f3'))
    df4 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > rd-tws*4) & (x['DAY'] <= rd-tws*3)]['SALES_VALUE'].sum()).reset_index(name ='f4'))
    df5 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > rd-tws*5) & (x['DAY'] <= rd-tws*4)]['SALES_VALUE'].sum()).reset_index(name ='f5'))
    df1['f2'] = df2['f2']
    df1['f3'] = df3['f3']
    df1['f4'] = df4['f4']
    df1['f5'] = df5['f5']   
    return df1.drop(columns = ['household_key'], inplace = False), df.drop(columns = ['household_key'], inplace = False)
    

In [25]:
X, y = get_dataset( rd = 697, tws = 14, ows = 14 )

In [26]:
# Solution
print('X.head(5)')
print(X.head(5))
print()
print('y.head(5)')
print(y.head(5))

X.head(5)
       f1     f2      f3     f4      f5
0  209.89  79.07   90.21  41.57  126.22
1    0.00   0.00  130.89   0.00    0.00
2    0.00  22.53    0.00   0.00   39.32
3    0.00   0.00    0.00   0.00    0.00
4   16.98   0.00    0.00   0.00    0.00

y.head(5)
   output_feature
0           81.29
1            0.00
2           36.38
3            0.00
4           12.99


## Task 3
Create a temporal dataset (pandas dataframe) that has:
* a Reference Date of day: MAX(day) - 7<br>
* a tumbling window of 7 days for five periods<br>
* `sum` as the tumbling window aggregate function<br>

The output feature should be a binary value:<br>
1 if the customer used a manufacturer coupon in the next seven days.<br>
0 otherwise.<br>

In [27]:
data

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2595727,1598,42305362535,711,92130,1,0.99,3228,0.00,1520,102,0.0,0.0
2595728,1598,42305362535,711,114102,1,8.89,3228,0.00,1520,102,0.0,0.0
2595729,1598,42305362535,711,133449,1,6.99,3228,0.00,1520,102,0.0,0.0
2595730,1598,42305362535,711,6923644,1,4.50,3228,-0.49,1520,102,0.0,0.0


In [49]:
def get_dataset(rd, tws, ows):
    #complete the code here
    data2 = data.groupby('household_key')
    df = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > rd) & (x['DAY'] <= rd+ows)]['COUPON_DISC'].sum()).reset_index(name ='output_feature'))
    df['output_feature'].loc[df['output_feature']<0]=1
    df1 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > rd-tws) & (x['DAY'] <= rd)]['SALES_VALUE'].sum()).reset_index(name ='f1'))
    df2 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > rd-tws*2) & (x['DAY'] <= rd-tws)]['SALES_VALUE'].sum()).reset_index(name ='f2'))
    df3 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > rd-tws*3) & (x['DAY'] <= rd-tws*2)]['SALES_VALUE'].sum()).reset_index(name ='f3'))
    df4 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > rd-tws*4) & (x['DAY'] <= rd-tws*3)]['SALES_VALUE'].sum()).reset_index(name ='f4'))
    df5 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > rd-tws*5) & (x['DAY'] <= rd-tws*4)]['SALES_VALUE'].sum()).reset_index(name ='f5'))
    df1['f2'] = df2['f2']
    df1['f3'] = df3['f3']
    df1['f4'] = df4['f4']
    df1['f5'] = df5['f5'] 
    return df1.drop(columns = ['household_key'], inplace = False), df.drop(columns = ['household_key'], inplace = False)


In [50]:
X, y = get_dataset( rd = 704, tws = 7, ows = 7)

/Users/qingxinmeng/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [52]:
# Solution
print('X.head(5)')
print(X.head(5))
print()
print('y.head(5)')
print(y.head(5))

X.head(5)
      f1      f2     f3   f4     f5
0   0.00  176.48  33.41  0.0  79.07
1   0.00    0.00   0.00  0.0   0.00
2  36.38    0.00   0.00  0.0  22.53
3   0.00    0.00   0.00  0.0   0.00
4  12.99   16.98   0.00  0.0   0.00

y.head(5)
   output_feature
0             1.0
1             0.0
2             0.0
3             0.0
4             0.0


# Part B
## Task 1
Setup a temporal hold-out evaluation.<br>
Set:
* output_window_size = 7
* tumbling_window_size = 7
* now = 711

And use a RandomForestRegressor as your predictor, n_estimators=100.<br>
Measure your error via the mean_absolute_error.<br>
Also report the mean MAE as a proportion of the mean of the output feature (it's typically more interpretable).

As a baseline, for each test time series, use the most recent input feature (NOTE: this is only valid since the output_window_size = tumbling_window_size).<br>

I.e if your test input features (as a dataframe) are called `test_X` and your most recent input feature is `f1` then you baseline predictions are simply `test_X.f1` for the former.

In [43]:
# Copy across your get_dataset function from Practical 4 Part A Task2b. It should have the following signiture:
# get_dataset(rd, tws, ows)
def get_dataset(rd, tws, ows):
    #complete the code here
    data2 = data.groupby('household_key')
    df = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > rd) & (x['DAY'] <= rd+ows)]['SALES_VALUE'].sum()).reset_index(name ='output_feature'))
    df1 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > rd-tws) & (x['DAY'] <= rd)]['SALES_VALUE'].sum()).reset_index(name ='f1'))
    df2 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > rd-tws*2) & (x['DAY'] <= rd-tws)]['SALES_VALUE'].sum()).reset_index(name ='f2'))
    df3 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > rd-tws*3) & (x['DAY'] <= rd-tws*2)]['SALES_VALUE'].sum()).reset_index(name ='f3'))
    df4 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > rd-tws*4) & (x['DAY'] <= rd-tws*3)]['SALES_VALUE'].sum()).reset_index(name ='f4'))
    df5 = pd.DataFrame(data2.apply(lambda x: x[(x['DAY'] > rd-tws*5) & (x['DAY'] <= rd-tws*4)]['SALES_VALUE'].sum()).reset_index(name ='f5'))
    df1['f2'] = df2['f2']
    df1['f3'] = df3['f3']
    df1['f4'] = df4['f4']
    df1['f5'] = df5['f5']   
    return df1.drop(columns = ['household_key'], inplace = False), df.drop(columns = ['household_key'], inplace = False)
    

In [44]:
output_window_size = 7 
tumbling_window_size = 7
now =  711

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np

#set up the model
model = RandomForestRegressor(n_estimators = 100)

test_X, test_y = get_dataset( now - output_window_size, tumbling_window_size, output_window_size )
train_X, train_y = get_dataset( now - 2*output_window_size, tumbling_window_size, output_window_size )


model.fit( train_X, train_y )
preds = model.predict(test_X)

s = mean_absolute_error(test_y, preds)
b = mean_absolute_error(test_y, test_X.f1)
b2 = mean_absolute_error(test_y, test_X.mean(axis = 1))
print('RF MAE: {}, Proportional MAE: {}'.format(s,s/np.mean(test_y)))
print('Baseline MAE: {}, Proportional MAE: {}'.format(b,b/np.mean(test_y)))
print('Baseline 2 MAE: {}, Proportional MAE 2: {}'.format(b2,b2/np.mean(test_y)))

/var/folders/jn/w4p7dcm52n7g22f03t3kdgbm0000gn/T/ipykernel_65384/530569131.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit( train_X, train_y )


RF MAE: 29.721938020255173, Proportional MAE: output_feature    0.84011
dtype: float64
Baseline MAE: 36.113916, Proportional MAE: output_feature    1.020784
dtype: float64
Baseline 2 MAE: 29.090495999999995, Proportional MAE 2: output_feature    0.822262
dtype: float64


# Task 2
Copy the above code below and modify it to do *repeated* temporal holdout.<br>
Keep all settings above the same, but add:
* total_holdout_sets = 5

In [45]:
output_window_size = 7 
tumbling_window_size = 7
total_holdout_sets = 5
now =  711

scores = []
basescores = []
basescores2 = []

model = RandomForestRegressor(n_estimators=100)

for i in range(total_holdout_sets):
    test_X, test_y = get_dataset( now - output_window_size, tumbling_window_size, output_window_size )
    train_X, train_y = get_dataset( now - 2*output_window_size, tumbling_window_size, output_window_size )
    model.fit( train_X, train_y )
    preds = model.predict(test_X)

    s = mean_absolute_error(test_y, preds)
    b = mean_absolute_error(test_y, test_X.f1)
    scores.append(s)
    basescores.append(b)
    now = now - output_window_size


print( np.mean(scores) / np.mean(test_y) )
print( np.mean(basescores) / np.mean(test_y) )

/var/folders/jn/w4p7dcm52n7g22f03t3kdgbm0000gn/T/ipykernel_65384/1103977551.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit( train_X, train_y )
/var/folders/jn/w4p7dcm52n7g22f03t3kdgbm0000gn/T/ipykernel_65384/1103977551.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit( train_X, train_y )
/var/folders/jn/w4p7dcm52n7g22f03t3kdgbm0000gn/T/ipykernel_65384/1103977551.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit( train_X, train_y )
/var/folders/jn/w4p7dcm52n7g22f03t3kdgbm0000gn/T/ipykernel_65384/1103977551.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape o

output_feature    0.87396
dtype: float64
output_feature    1.022619
dtype: float64


## Task 3
Setup a repeated temporal hold-out evaluation, repeating 
* output_window_size = 14
* tumbling_window_size = 14
* total_holdout_sets = 10
* now = 711

And use a RandomForestRegressor as your predictor, with n_estimators=100.

In [ ]:
output_window_size = 14 
tumbling_window_size = 14
total_holdout_sets = 10
now =  711

scores = []
basescores = []

model = RandomForestRegressor(n_estimators=100)

for i in range(total_holdout_sets):
    test = get_dataset( now - output_window_size, tumbling_window_size, output_window_size )
    train = get_dataset( now - 2*output_window_size, tumbling_window_size, output_window_size )
    model.fit( train[0], train[1] )
    preds = model.predict(test[0])

    s = mean_absolute_error(test[1], preds)
    b = mean_absolute_error(test[1], test[0].f1)
    scores.append(s)
    basescores.append(b)
    now = now - output_window_size

    

print( np.mean(scores) / np.mean(test[1]) )
print( np.mean(basescores) / np.mean(test[1]) )

/var/folders/jn/w4p7dcm52n7g22f03t3kdgbm0000gn/T/ipykernel_65384/1198337962.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit( train[0], train[1] )
/var/folders/jn/w4p7dcm52n7g22f03t3kdgbm0000gn/T/ipykernel_65384/1198337962.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit( train[0], train[1] )
/var/folders/jn/w4p7dcm52n7g22f03t3kdgbm0000gn/T/ipykernel_65384/1198337962.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit( train[0], train[1] )
/var/folders/jn/w4p7dcm52n7g22f03t3kdgbm0000gn/T/ipykernel_65384/1198337962.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s